## Introducción

En el presente trabajo se propone realizar un análisis de los datos recolectados por una empresa en los últimos 2 meses sobre los clientes que fueron a ver la pelicula Frozen 3. La encuesta consistía en una serie de preguntas personales que concluía si el cliente volvería a ese cine si salía la secuela, Frozen 4.

El objetivo de la empresa es poder dirigir futuras campañas de marketing digital en base a esta información recolectada.

## Objetivos

El objetivo de este notebook es:
*   Entender los datos
*   Ver cómo los datos se relacionan entre si
*   Poder sacar alguna conclusión o descubrir un patrón a partir de estos
*   Indicar cuáles son los factores más importantes que determinan si un usuario va a ir al cine a ver Frozen 4 o no.
*   Poder llegar a armar un baseline

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from math import pi

## Carga y limpieza del set de datos

In [7]:
usuario_volveria_df = pd.read_csv(r'C:\Users\Enrique\Desktop\analisis\Datasets\tp-2020-2c-train-cols1.csv')
usuario_volveria_df.head()

,id_usuario,volveria
0,117,0
1,658,0
2,794,0
3,455,0
4,173,1


In [9]:
info_fiumark_df = pd.read_csv(r'C:\Users\Enrique\Desktop\analisis\Datasets\tp-2020-2c-train-cols2.csv')
info_fiumark_df.head()

,tipo_de_sala,nombre,id_usuario,genero,edad,amigos,parientes,id_ticket,precio_ticket,fila,nombre_sede
0,4d,Señor Camilo Pedro,117,hombre,73.5,0,0,59258;,1,NaN,fiumark_quilmes
1,4d,Señora Raquel Angelica,658,mujer,35.0,1,1,586:6;,2,NaN,fiumark_quilmes
2,normal,Señor Antonio Federico,794,hombre,NaN,0,0,"RE""39822",3,NaN,fiumark_chacarita
3,4d,Señor Osvaldo Aureliano,455,hombre,NaN,0,0,"C17""4:39",1,NaN,fiumark_palermo
4,4d,Señorita Rita Eudosia,173,mujer,4.0,1,1,569964,2,NaN,fiumark_palermo


In [10]:
usuario_volveria_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 801 entries, 0 to 800
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   id_usuario  801 non-null    int64
 1   volveria    801 non-null    int64
dtypes: int64(2)
memory usage: 12.6 KB


Notamos que la columna de 'volveria' es de tipo int64, cuando solamente tiene valores que pueden ser 0 o 1. Un mejor tipo de dato para este caso puede ser un int8. Esto permitiría ahorrar memoria. (Aunque en este caso no sean muchos datos)

In [ ]:
usuario_volveria_df['volveria'] = usuario_volveria_df['volveria'].astype(np.int8)